<a href="https://colab.research.google.com/github/chi-hun/dacon/blob/main/%EC%87%BC%ED%95%91%EB%AA%B0_%EB%A6%AC%EB%B7%B0_%ED%8F%89%EC%A0%90_%EB%B6%84%EB%A5%98_%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

쇼핑몰 리뷰 평점 분류 경진대회 https://dacon.io/competitions/official/235938/overview/description

# module

In [2]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 453 kB 46.5 MB/s 


In [4]:
!pip install git+https://github.com/ssut/py-hanspell.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-8ydtg90c
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-8ydtg90c
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=5a92a23875866889036ad90fc57146378ba71a5b98fb65e46719f509bfd2795c
  Stored in directory: /tmp/pip-ephem-wheel-cache-odkhwikh/wheels/ab/f5/7b/d4124bb329c905301baed80e2ae45aa14e824f62ebc3ec2cc4
Successfully built py-hanspell


In [110]:
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from hanspell import spell_checker
import re
import tqdm

# EDA

In [123]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       25000 non-null  int64 
 1   reviews  25000 non-null  object
 2   target   25000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 586.1+ KB


In [8]:
train.head()

,id,reviews,target
0,0,조아요 처음구입 싸게햇어요,2
1,1,생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천원가량 주고 사기 너무 ...,1
2,2,디자인은괜찮은데 상품이 금이가서 교환했는데 두번째받은상품도 까져있고 안쪽에 금이가져...,2
3,3,기전에 이 제품말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다. 샘...,2
4,4,튼튼하고 손목을 잘 받쳐주네요~,5


In [9]:
train[train['target'] == 5].head()

,id,reviews,target
4,4,튼튼하고 손목을 잘 받쳐주네요~,5
6,6,타이밍이 좋았는지 배송도 빠르고 맛있었어요! 크기가 두가지가 있길래 큰걸 사봤는데 ...,5
8,8,재구매 평소 사용하던 제품이에요. 한번에 쟁여두려 여러개 삽니다 .,5
10,10,재구매 항상 여기서만 주문해서 잘 쓰고있어요 후딱쓰기좋아요 크기도 좋구,5
12,12,재구매 좋은제품 저렴하게 잘 구매했습니다.,5


In [10]:
train[train['target'] == 1].head()

,id,reviews,target
1,1,생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천원가량 주고 사기 너무 ...,1
19,19,완전 골아서 왔어요 집에서 썩은내가 진동해서 베란다문 열고 그래도 내장이랑 분리 해...,1
40,40,이상한 모양이 찍혀 나와요 제 프린터의 문제인줄 알았는데 다른 사람도 똑같은 모양이...,1
41,41,양이 너무 적어요 밖에서 사먹는게 더 쌀듯,1
42,42,양념통5개 오일병5개 주문했는데 양념통 뚜껑하나가 스텐마감이 안되어있어서 검지손가락...,1


In [12]:
test.head()

,id,reviews
0,0,채소가 약간 시들어 있어요
1,1,발톱 두껍고 단단한 분들 써도 소용없어요 이 테이프 물렁거리고 힘이없어서 들어 올리...
2,2,부들부들 좋네요 입어보고 시원하면 또 살게요
3,3,이런 1. 8 골드 주라니깐 파란개 오네 회사전화걸어도 받지도 않고 머하자는거임?
4,4,검수도 없이 보내구 불량 배송비 5000원 청구하네요 완전별로 별하나도 아까워요


In [89]:
train.shape, test.shape

((25000, 3), (25000, 2))

## spell_checker

In [85]:
train1 = train.iloc[:10000,:]

In [90]:
def spellchecker(x):
    x = re.sub('\W', ' ', x)
    sc = spell_checker.check(x).checked
    return sc
train1['reviews'] = train1['reviews'].apply(spellchecker)
train1.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,id,reviews,target
0,0,좋아요 처음 구입 싸게 했어요,2
1,1,생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천 원가량 주고 사기 너무...,1
2,2,디자인은 괜찮은데 상품이 금이 가서 교환했는데 두 번째 받은 상품도 까져있고 안쪽에...,2
3,3,기전에 이 제품 말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다 ...,2
4,4,튼튼하고 손목을 잘 받쳐주네요,5


In [ ]:
test['reviews'] = test['reviews'].apply(spellchecker)

In [106]:
train2 = train.iloc[10000:20000,:]

In [107]:
def spellchecker(x):
    x = re.sub('\W', ' ', x)
    sc = spell_checker.check(x).checked
    return sc
train2['reviews'] = train2['reviews'].apply(spellchecker)
train2.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,id,reviews,target
10000,10000,완전별로요,2
10001,10001,요거 쓰고 나면 다른 제품은 만족스럽지가 않아요ㅎㅎ 그만큼 깨끗하고 락스 냄새도 안...,5
10002,10002,재구매 매번 구매하고 있는 제 빠른 배송과 착한 가격 너무나도 잘 이용하고 있습니다...,5
10003,10003,굳굳 좋아요 잘 쓰는 중,2
10004,10004,엄청 맛있진 않고 쏘소인 거 같아요,2


In [108]:
train3 = train.iloc[20000:,:]

In [109]:
def spellchecker(x):
    x = re.sub('\W', ' ', x)
    sc = spell_checker.check(x).checked
    return sc
train3['reviews'] = train3['reviews'].apply(spellchecker)
train3.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,id,reviews,target
20000,20000,신발정리하기 좋킨 한데 부실해요,2
20001,20001,너무싸구려틱해요ㆍ,2
20002,20002,너무 약해서 옷을 걸을 때마다 흔들거려 쓰러질 거 같아요 디자인은 괜찮아요,2
20003,20003,배송이 엉망 확인에는 배달 완료로 뜨는데 배송은 안되고 기사님 전화는 안 받고 기다...,1
20004,20004,재구매 사진 첨부가 안 되네요 저번에도 여기서 구입했어요,1


In [124]:
def spellchecker(x):
    x = re.sub('\W', ' ', x)
    sc = spell_checker.check(x).checked
    return sc
for i in tqdm.tqdm(range(train.shape[0])):
    train.loc[i,'reviews_ck'] = spellchecker(train.loc[i,'reviews'])
for i in tqdm.tqdm(range(test.shape[0])):
    test.loc[i,'reviews_ck'] = spellchecker(test.loc[i,'reviews'])

 45%|████▍     | 11175/25000 [20:14<25:02,  9.20it/s]


JSONDecodeError: ignored

In [139]:
def spellchecker(x):
    x = re.sub('\W', ' ', x)
    sc = spell_checker.check(x).checked
    return sc

a, b = 0, 1000
for i in tqdm.tqdm(range(25)):
    train.loc[a:b,'reviews_ck'] = train.loc[a:b,'reviews'].apply(spellchecker)
    a += 1000
    b += 1000
# for i in tqdm.tqdm(range(test.shape[0])):
#     test.loc[i,'reviews_ck'] = spellchecker(test.loc[i,'reviews'])

 76%|███████▌  | 19/25 [33:11<10:28, 104.79s/it]


KeyboardInterrupt: ignored

In [138]:
train.loc[24999:25000]

,id,reviews,target,reviews_ck
24999,24999,호호애미 제품은 늘 믿고 써요,5,NaN


In [ ]:
okt = Okt()
text_o = okt.nouns(text)
text_o = [i for i in text_o if len(i) > 1]